In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

### Contributors Data Cleaning

In [2]:
data_path = Path("../") / "data"

if previous_contrib_data_list := [
    pd.read_feather(contrib_file)
    for contrib_file in data_path.glob("contributors*.feather")
]:
    cons = pd.concat(previous_contrib_data_list, ignore_index=True)

In [3]:
cons

,original_index,url,status,contributors
0,17532,https://api.github.com/repos/WinMin/4616577921...,success,"{'avatar_url': None, 'block': None, 'contribut..."
1,17534,https://api.github.com/repos/Cossack9989/fa971...,success,"{'avatar_url': None, 'block': None, 'contribut..."
2,17537,https://api.github.com/repos/Cossack9989/Vulns,success,{'avatar_url': 'https://avatars.githubusercont...
3,19242,https://api.github.com/repos/jaygreig86/dmitry,success,{'avatar_url': 'https://avatars.githubusercont...
4,19276,https://api.github.com/repos/tendenci/tendenci,success,{'avatar_url': 'https://avatars.githubusercont...
...,...,...,...,...
58650,1023363,https://api.github.com/repos/ipxe/ipxe,success,{'avatar_url': 'https://avatars.githubusercont...
58651,1023363,https://api.github.com/repos/ipxe/ipxe,success,{'avatar_url': 'https://avatars.githubusercont...
58652,1023363,https://api.github.com/repos/ipxe/ipxe,success,{'avatar_url': 'https://avatars.githubusercont...
58653,1023364,https://api.github.com/repos/IthacaLabs/Parallels,success,{'avatar_url': 'https://avatars.githubusercont...


In [4]:
result = []
for i in range(len(cons)):
    con = cons.contributors[i]
    if('login' in con):
        name = con['login']
        result.append([cons.original_index[i], cons.url[i], name])

In [5]:
loginsdf = pd.DataFrame(result, columns = ['original_index', 'url', 'logins'])
loginsdf

,original_index,url,logins
0,17532,https://api.github.com/repos/WinMin/4616577921...,None
1,17534,https://api.github.com/repos/Cossack9989/fa971...,None
2,17537,https://api.github.com/repos/Cossack9989/Vulns,Cossack9989
3,19242,https://api.github.com/repos/jaygreig86/dmitry,jaygreig86
4,19276,https://api.github.com/repos/tendenci/tendenci,jennyq
...,...,...,...
58650,1023363,https://api.github.com/repos/ipxe/ipxe,kevintuantran
58651,1023363,https://api.github.com/repos/ipxe/ipxe,mhaeuser
58652,1023363,https://api.github.com/repos/ipxe/ipxe,pkt
58653,1023364,https://api.github.com/repos/IthacaLabs/Parallels,IthacaLabs


In [6]:
loginsdf = loginsdf[loginsdf.logins.notnull()]
loginsdf

,original_index,url,logins
2,17537,https://api.github.com/repos/Cossack9989/Vulns,Cossack9989
3,19242,https://api.github.com/repos/jaygreig86/dmitry,jaygreig86
4,19276,https://api.github.com/repos/tendenci/tendenci,jennyq
5,112255,https://api.github.com/repos/ImpulseAdventure/...,ImpulseAdventure
6,112264,https://api.github.com/repos/matrixssl/matrixssl,matrixssl-admin
...,...,...,...
58650,1023363,https://api.github.com/repos/ipxe/ipxe,kevintuantran
58651,1023363,https://api.github.com/repos/ipxe/ipxe,mhaeuser
58652,1023363,https://api.github.com/repos/ipxe/ipxe,pkt
58653,1023364,https://api.github.com/repos/IthacaLabs/Parallels,IthacaLabs


In [7]:
loginsdf = loginsdf.drop_duplicates()
loginsdf

,original_index,url,logins
2,17537,https://api.github.com/repos/Cossack9989/Vulns,Cossack9989
3,19242,https://api.github.com/repos/jaygreig86/dmitry,jaygreig86
4,19276,https://api.github.com/repos/tendenci/tendenci,jennyq
5,112255,https://api.github.com/repos/ImpulseAdventure/...,ImpulseAdventure
6,112264,https://api.github.com/repos/matrixssl/matrixssl,matrixssl-admin
...,...,...,...
58650,1023363,https://api.github.com/repos/ipxe/ipxe,kevintuantran
58651,1023363,https://api.github.com/repos/ipxe/ipxe,mhaeuser
58652,1023363,https://api.github.com/repos/ipxe/ipxe,pkt
58653,1023364,https://api.github.com/repos/IthacaLabs/Parallels,IthacaLabs


In [8]:
loginsdf.reset_index().to_feather('../data/github_usernames_final.feather')

### Languages Data Cleaning

In [9]:
data_path = Path("../") / "data"

if previous_langs_data_list := [
    pd.read_feather(lang_file)
    for lang_file in data_path.glob("languages*.feather")
]:
    langs = pd.concat(previous_langs_data_list, ignore_index=True)

In [10]:
langs

,original_index,url,status,languages
0,402641,https://api.github.com/repos/omniauth/omniauth...,success,"{'ANTLR': None, 'APL': None, 'ASP': None, 'ASP..."
1,402649,https://api.github.com/repos/esotalk/esoTalk,success,"{'ANTLR': None, 'APL': None, 'ASP': None, 'ASP..."
2,402660,https://api.github.com/repos/grymer/CVE,default,"{'ANTLR': None, 'APL': None, 'ASP': None, 'ASP..."
3,402964,https://api.github.com/repos/braekling/WP-Matomo,success,"{'ANTLR': None, 'APL': None, 'ASP': None, 'ASP..."
4,403323,https://api.github.com/repos/scaron/prettyphoto,success,"{'ANTLR': None, 'APL': None, 'ASP': None, 'ASP..."
...,...,...,...,...
10220,507806,https://api.github.com/repos/cseasholtz/CVE-20...,success,"{'1C Enterprise': None, 'AGS Script': None, 'A..."
10221,507808,https://api.github.com/repos/cseasholtz/CVE-20...,default,"{'1C Enterprise': None, 'AGS Script': None, 'A..."
10222,507830,https://api.github.com/repos/sharkdp/bat,success,"{'1C Enterprise': None, 'AGS Script': None, 'A..."
10223,507841,https://api.github.com/repos/nightscout/cgm-re...,success,"{'1C Enterprise': None, 'AGS Script': None, 'A..."


In [11]:
result = []
for i in range(len(langs)):
    lang = langs.languages[i]
    for key, value in lang.items():
        if value is not None:
            result.append([langs.original_index[i], langs.url[i], key, str(value)])

In [12]:
langsdf = pd.DataFrame(result, columns = ['original_index', 'url', 'language', 'ratio'])
langsdf

,original_index,url,language,ratio
0,402641,https://api.github.com/repos/omniauth/omniauth...,Ruby,1076.0
1,402649,https://api.github.com/repos/esotalk/esoTalk,CSS,46680.0
2,402649,https://api.github.com/repos/esotalk/esoTalk,JavaScript,252946.0
3,402649,https://api.github.com/repos/esotalk/esoTalk,PHP,870210.0
4,402964,https://api.github.com/repos/braekling/WP-Matomo,CSS,794.0
...,...,...,...,...
41758,507841,https://api.github.com/repos/nightscout/cgm-re...,Procfile,31.0
41759,507841,https://api.github.com/repos/nightscout/cgm-re...,Shell,6453.0
41760,507881,https://api.github.com/repos/dankogai/p5-encode,C,54177.0
41761,507881,https://api.github.com/repos/dankogai/p5-encode,Perl,429663.0


In [13]:
langsdf = langsdf.drop_duplicates()
langsdf

,original_index,url,language,ratio
0,402641,https://api.github.com/repos/omniauth/omniauth...,Ruby,1076.0
1,402649,https://api.github.com/repos/esotalk/esoTalk,CSS,46680.0
2,402649,https://api.github.com/repos/esotalk/esoTalk,JavaScript,252946.0
3,402649,https://api.github.com/repos/esotalk/esoTalk,PHP,870210.0
4,402964,https://api.github.com/repos/braekling/WP-Matomo,CSS,794.0
...,...,...,...,...
41758,507841,https://api.github.com/repos/nightscout/cgm-re...,Procfile,31.0
41759,507841,https://api.github.com/repos/nightscout/cgm-re...,Shell,6453.0
41760,507881,https://api.github.com/repos/dankogai/p5-encode,C,54177.0
41761,507881,https://api.github.com/repos/dankogai/p5-encode,Perl,429663.0


In [14]:
langsdf.reset_index().to_feather('../data/github_langs_final.feather')